##### Limpieza de datos del archivo de contaminacion
Eliminacion de columnas 'PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'source_file'.

Mirar si hay fechas repetidas. (NO)

In [2]:
import pandas as pd
import os
path = os.getcwd()
os.makedirs(path + "/DatosLimpios", exist_ok=True)
fichero = "https://raw.githubusercontent.com/albercol/TFG/main/Fase%201.3/PlazaEliptica/ContaminacionPlazaEliptica.csv"
df =  pd.read_csv(fichero,encoding="UTF-8")
df = df.drop(['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'source_file', 'V01','V02', 'V03','V04',
             'V05','V06','V07','V08', 'V09','V10','V11', 'V12', 'V13','V14', 'V15','V16','V17','V18','V19','V20',
             'V21','V22','V23', 'V24'], axis=1)

df.sort_values(['ANO','MES','DIA', 'MAGNITUD'], inplace=True)

#convertimos las horas en una unica columna.
df1 = pd.melt(df, id_vars =['DIA','MES','ANO','MAGNITUD'], var_name= "HORA")
#Ordenamos por magnitud.
df1.sort_values(['ANO','MES','DIA', 'MAGNITUD'], inplace=True)
#Convertimos las magnitudes en distintas columnas.
df2 = pd.pivot_table(df1, index=['DIA','MES','ANO','HORA'], columns=["MAGNITUD"],values='value').reset_index()
#Renombramos las columnas.
df2.set_axis(['DIA', 'MES', 'ANO', 'HORA', 'CO' , 'NO', 'NO2', 'PM2.5', 'PM10', 'NOx', 'O3'], axis='columns', inplace=True)

#Transformamos el formato hora a numeros enteros
df2['HORA'] = df2['HORA'].str.extract('(\d+)', expand=False)

#convertimos las la hora y los cintaminantes posibles a enteros
magnitudes = ['HORA','NO','NO2','NOx']
for c in magnitudes:
    df2[c] = df2[c].astype(int) 

#Ordenamos por año, mes y dia
df2.sort_values(['ANO', 'MES', 'DIA', 'HORA'], inplace=True)

#Guardamos en .CSV
df2.to_csv(path+"/DatosLimpios/"+"Contaminacion.csv", header=True, index=False, encoding="utf-8")

#### Limpieza de datos del archivo de Meteorología.
Eliminacion de columnas ('PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'source_file') (SI)

Mirar si hay fechas repetidas. (NO)

In [6]:
import pandas as pd
import os
path = os.getcwd()
os.makedirs(path + "/DatosLimpios", exist_ok=True)
fichero = "https://raw.githubusercontent.com/albercol/TFG/main/Fase%201.3/PlazaEliptica/MeteorologiaPlazaEliptica.csv"
df =  pd.read_csv(fichero,encoding="UTF-8")
df = df.drop(['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'source_file', 'V01','V02', 'V03','V04',
             'V05','V06','V07','V08', 'V09','V10','V11', 'V12', 'V13','V14', 'V15','V16','V17','V18','V19','V20',
             'V21','V22','V23', 'V24'], axis=1)

df.sort_values(['ANO','MES','DIA', 'MAGNITUD'], inplace=True)

#convertimos las horas en una unica columna.
df1 = pd.melt(df, id_vars =['DIA','MES','ANO','MAGNITUD'], var_name= "HORA")
#Ordenamos por magnitud.
df1.sort_values(['ANO','MES','DIA', 'MAGNITUD'], inplace=True)
#Convertimos las magnitudes en distintas columnas.
df2 = pd.pivot_table(df1, index=['DIA','MES','ANO','HORA'], columns=["MAGNITUD"] ,values='value').reset_index()
df2.set_axis(['DIA', 'MES', 'ANO', 'HORA', 'VELOCIDAD VIENTO' , 'DIR. VIENTO', 'TEMPERATURA', 'HUMEDAD RELATIVA', 'PRESION BARIOMETRICA', 'PRECIPITACIONES'], axis='columns', inplace=True)
#Transformamos el formato hora a numeros enteros
df2['HORA'] = df2['HORA'].str.extract('(\d+)', expand=False)

magnitudes = ['HORA', 'HUMEDAD RELATIVA', 'PRESION BARIOMETRICA']
for c in magnitudes:
    df2[c] = df2[c].astype(int) 
    
#Ordenamos por año, mes y dia
df2.sort_values(['ANO', 'MES', 'DIA', 'HORA'], inplace=True)

#Guardamos en .CSV
df2.to_csv(path+"/DatosLimpios/"+"Meteorologia.csv", header=True, index=False, encoding="utf-8")

#### Limpieza de datos del archivo de trafico.
Separar la fecha en tres columnas.(SI)

Mirar si hay fechas repetidas. (NO)

In [7]:
import pandas as pd
import os
path = os.getcwd()
os.makedirs(path + "/DatosLimpios", exist_ok=True)

#Separamos la fecha en tres columnas y eliminamos la columna 'FDIA'
fichero = "https://raw.githubusercontent.com/albercol/TFG/main/Fase%201.3/PlazaEliptica/TraficoPlazaEliptica.csv"
df =  pd.read_csv(fichero,encoding="UTF-8")
df['FDIA'] = pd.to_datetime(df['FDIA'], dayfirst=True)
df.insert(2, 'DIA', df.mean(1))
df.insert(2, 'MES', df.mean(1))
df.insert(2, 'ANO', df.mean(1))
df['DIA']=df['FDIA'].dt.day
df['MES']=df['FDIA'].dt.month
df['ANO']=df['FDIA'].dt.year
df = df.drop(['FDIA'], axis=1)

freq = df.groupby(['ANO', 'MES', 'DIA']).size()

#Ordenamos por año, mes dia y estacion.
df.sort_values(['ANO','MES','DIA', 'FEST'], inplace=True)
df 
#convertimos las horas en una unica columna.
df1 = pd.melt(df, id_vars =['DIA','MES','ANO','FEST'], var_name= "HORA")
#Ordenamos por magnitud.
df1.sort_values(['ANO','MES','DIA', 'FEST'], inplace=True)
#Convertimos las magnitudes en distintas columnas.
df2 = pd.pivot_table(df1, index=['DIA','MES','ANO','HORA'], columns=["FEST"] ,values='value').reset_index()
#Transformamos el formato hora a numeros enteros
df2['HORA'] = df2['HORA'].str.extract('(\d+)', expand=False)

magnitudes = ['HORA', 'ES10', 'ES53', 'ES54']
for c in magnitudes:
    df2[c] = df2[c].astype(int) 

#Ordenamos por año mes y dia
df2.sort_values(['ANO', 'MES', 'DIA', 'HORA'], inplace=True)
#Guardamos en .CSV
df2.to_csv(path+"/DatosLimpios/"+"Trafico.csv", header=True, index=False, encoding="utf-8")

C:\Users\Alberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Alberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Alberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Alberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Dropping of nuisance columns in D